# Getting Started with ❄️ Anthropic on Snowflake Cortex

Build an intelligent question-answering system using Anthropic's Claude and Snowflake's AI capabilities.⚡️

This notebook demonstrates how to build an end-to-end application that:
1. Processes PDF documents using Cortex Process Docouments
2. Creates Cortex Search Service to do keyword and vector searches
3. Implements a chat interface using Snowflake's Cortex and Anthropic's Claude in Streamlit

Check out the [Quickstart](https://quickstarts.snowflake.com/guide/getting_started_on_anthropic_with_snowflake_cortex) for instructions on getting setup for this Notebook

## Setting Up Your Environment 🎒

First, we'll import the required packages and set up our Snowflake session. The notebook uses several key packages:
- `streamlit`: For creating the interactive chat interface
- `snowflake-ml-python`: For Snowflake Cortex for embeddings and LLM capabilities


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import json

from snowflake.snowpark.context import get_active_session
from snowflake.cortex import complete, EmbedText768
from snowflake.snowpark.types import VectorType, FloatType
from snowflake.core.table import Table, TableColumn
from snowflake.core import CreateMode, Root
from snowflake.snowpark.functions import cast, col


session = get_active_session()
current_warehouse = session.get_current_warehouse()
database_name = session.get_current_database()
schema_name = session.get_current_schema()
role_name = session.get_current_role()
service_name = 'document_search_service'
root = Root(session)
database = root.databases[database_name]
schema = database.schemas[schema_name]


## Setting Up Stage Variables 📁

We'll define our stage name and retrieve the list of files to process. This stage should contain the PDF documents we want to analyze.

In [ ]:
stage_name="@Documents"
files = session.sql(f"LIST{stage_name}").collect()

## Document Processing Functions 📄

We'll create functions to extract text from PDF files


In [ ]:
def process(file_name: str):
    query = """
        SELECT TO_VARCHAR(
            SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
                ?,
                ?,
                {'mode': 'OCR'}):content
        ) AS OCR;
    """

    resp = session.sql(query, params=[stage_name, file_name]).collect()
    text = resp[0]['OCR']
    
    df = pd.DataFrame({
        'TEXT' : [text],
        'FILE_NAME': file_name
    })
    
    return df

## Processing Documents

Now we'll:
1. Process all documents in our stage
2. Store the results in our table

In [ ]:
# Extract file names and process files
file_names = [file['name'].split('/')[1] for file in files]

# Download and process files into a DataFrame
final_dataframe = pd.concat([
    process(file_name)
    for file_name in file_names
], ignore_index=True)

snowpark_df = session.create_dataframe(final_dataframe).select(
    col("file_name"),
    col("text")
)

# Write the transformed data directly to the target table
snowpark_df.write.mode("overwrite").save_as_table("docs_text_table")

## Create Cortex Search Service 

### Key Components Explained 📚

#### Required Parameters

- `ON`: Specifies the column containing the text to be indexed  
- `ATTRIBUTES`: Additional columns to include in search results (e.g., file\_name)  
- `WAREHOUSE`: Compute warehouse for processing the embeddings  
- `TARGET_LAG`: Maximum allowed lag for index updates  
- `EMBEDDING_MODEL`: Model used to generate text embeddings  
- Source query: The SELECT statement defining the data to index

#### Configuration Options 🔧

1. Target Lag Settings:  
     
   - Shorter lag times mean more frequent updates  
   - Common values: '1 hour', '1 day', '1 week'  
   - Balance freshness needs with compute costs

   

2. Embedding Model Options:  
     
   - 'snowflake-arctic-embed-l-v2.0': Latest Snowflake embedding model  
   - Optimized for English language content  
   - 384-dimensional embeddings

   

3. Warehouse Considerations:  
     
   - Choose size based on data volume  
   - Consider compute costs vs update frequency  
   - Monitor warehouse utilization

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE {{service_name}}
  ON text
  ATTRIBUTES file_name
  WAREHOUSE = {{current_warehouse}}
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
        text,
        file_name
    FROM docs_text_table
);

## Building the Chat Interface 💬

Finally, we'll create our chat interface that uses:
- Utilizes the Cortex Search Service for finding relevant context
- Chat history management for conversation continuity
- Anthropic's Claude model for generating responses
- Streamlit for the user interface

Key parameters:
- `num_results`: Number of context results provided (default: 3)
- `model_name`: Language model used (default: "claude-haiku-4-5")
- `history_length`: Chat history length (default: 5)

In [ ]:
import streamlit as st
import json
import time
import re
import pandas as pd
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col
from snowflake.cortex import complete
from datetime import datetime

# Snowflake session setup
session = get_active_session()
root = Root(session)
database_name = session.get_current_database()
schema_name = session.get_current_schema()
service_name = 'document_search_service'

# Configuration
model_name = "claude-haiku-4-5"
num_results = 3
history_length = 5
DEFAULT_CARBON_PRICE = 78.54  # Current market price as of Oct 31, 2025
MIN_REQUEST_INTERVAL = 2.0
RETRY_DELAY = 5

# Recent EU ETS prices from October 2025
# Source: Trading Economics (tradingeconomics.com/commodity/carbon)
RECENT_PRICES = {
    "2025-10-31": 78.54,
    "2025-10-01": 76.30,
    "2025-09-15": 74.80,
    "2025-09-01": 73.20,
}

PRICE_SOURCE_URL = "https://tradingeconomics.com/commodity/carbon"

DEFAULT_EMISSIONS = {
    "steel": 2.3,
    "aluminum": 8.6,
    "cement": 0.9,
    "fertilizer": 1.5,
    "electricity": 0.4,
    "glass": 0.8,
    "ceramics": 0.7,
    "hydrogen": 10.0,
}

def write_price_to_stage(carbon_price, is_manual=False):
    """Write carbon price to a .txt file and upload to @Documents stage."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    source_type = "Manual override" if is_manual else "Default market price"
    
    content = f"""Live EU ETS Carbon Price: €{carbon_price:.2f}/tonne CO₂
Updated: {timestamp}
Source: {source_type}

This price represents the current European Union Emissions Trading System (EU ETS) allowance price.
Use this for CBAM cost calculations unless overridden by official EU guidance.

CBAM Formula:
CBAM Cost = (Embedded Emissions in tCO₂e) × (EU ETS Price - Carbon Price Paid at Origin)

Note: Prices fluctuate daily. Update regularly for accurate calculations.
"""
    
    try:
        with open("/tmp/live_carbon_price.txt", "w") as f:
            f.write(content)
        session.file.put("/tmp/live_carbon_price.txt", "@Documents", overwrite=True, auto_compress=False)
        return True
    except Exception as e:
        st.error(f"Failed to write price: {str(e)}")
        return False

def process(file_name: str):
    """Process document and extract text using OCR."""
    try:
        query = """
            SELECT TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT(?, ?, {'mode': 'OCR'}):content) AS OCR;
        """
        resp = session.sql(query, params=["@Documents", file_name]).collect()
        text = resp[0]['OCR']
        return pd.DataFrame({'TEXT': [text], 'FILE_NAME': file_name})
    except Exception as e:
        return pd.DataFrame({'TEXT': [''], 'FILE_NAME': file_name})

def reindex_documents():
    """Reindex all documents in the stage - only on first load."""
    try:
        files = session.sql("LIST @Documents").collect()
        file_names = [file['name'].split('/')[-1] for file in files]
        
        if not file_names:
            return False
        
        dataframes = []
        for name in file_names:
            df = process(name)
            if not df.empty and df['TEXT'].iloc[0]:
                dataframes.append(df)
        
        if dataframes:
            final_dataframe = pd.concat(dataframes, ignore_index=True)
            snowpark_df = session.create_dataframe(final_dataframe).select(col("FILE_NAME"), col("TEXT"))
            snowpark_df.write.mode("overwrite").save_as_table("docs_text_table")
            return True
        return False
    except:
        return False

def init_messages():
    """Initialize session state variables."""
    if "messages" not in st.session_state:
        st.session_state.messages = []
    if "last_request_time" not in st.session_state:
        st.session_state.last_request_time = 0
    if "carbon_price" not in st.session_state:
        st.session_state.carbon_price = DEFAULT_CARBON_PRICE
    if "is_manual_price" not in st.session_state:
        st.session_state.is_manual_price = False
    if "selected_historic_date" not in st.session_state:
        st.session_state.selected_historic_date = None
    if "manual_override_price" not in st.session_state:
        st.session_state.manual_override_price = None
    if "initialized" not in st.session_state:
        st.session_state.initialized = False
        # Only reindex on first load
        write_price_to_stage(DEFAULT_CARBON_PRICE, False)
        reindex_documents()
        st.session_state.initialized = True

def init_config_options():
    """Initialize UI configuration options."""
    st.session_state.num_chat_messages = history_length
    
    # Top row: Price display, reset, and clear chat
    col1, col2, col3 = st.columns([3, 1, 1])
    
    with col1:
        # Always show current market price (never changes)
        st.metric(
            "Current EU ETS Price", 
            f"€{DEFAULT_CARBON_PRICE:.2f}/tCO₂e"
        )
        # Show date stamp for default price
        if st.session_state.manual_override_price is None and st.session_state.selected_historic_date is None:
            st.caption("Using default price - Updated: 2024-11-01")
        
        # Show alert when manual price is active
        if st.session_state.manual_override_price is not None:
            st.caption(f"⚠️ Calculator is using manually entered price: €{st.session_state.manual_override_price:.2f}")
        
        # Show alert when historic price is active
        elif st.session_state.selected_historic_date is not None:
            historic_price = RECENT_PRICES[st.session_state.selected_historic_date]
            st.caption(f"⚠️ Calculator is using historical price: €{historic_price:.2f}")
    
    with col2:
        # Reset button - always visible when not using default
        if st.session_state.manual_override_price is not None or st.session_state.selected_historic_date is not None:
            if st.button("🔄 Reset", use_container_width=True):
                st.session_state.carbon_price = DEFAULT_CARBON_PRICE
                st.session_state.manual_override_price = None
                st.session_state.selected_historic_date = None
                write_price_to_stage(DEFAULT_CARBON_PRICE, False)
                st.rerun()
    
    with col3:
        if st.button("🗑️ Clear Chat", use_container_width=True):
            st.session_state.messages = []
            st.rerun()
    
    # Price update section - always expanded
    st.markdown("---")
    st.subheader("💶 Update Carbon Price")
    st.caption("Source: ICE Futures Europe & EEX (European Energy Exchange)")
    
    # Historic price buttons in a row
    st.write("**Historical Prices:**")
    cols = st.columns(len(RECENT_PRICES))
    
    for idx, (date, price) in enumerate(sorted(RECENT_PRICES.items(), reverse=True)):
        with cols[idx]:
            is_selected = st.session_state.selected_historic_date == date
            button_type = "primary" if is_selected else "secondary"
            
            if st.button(
                f"{date}\n€{price:.2f}", 
                key=f"price_{date}", 
                use_container_width=True,
                type=button_type
            ):
                st.session_state.carbon_price = price
                st.session_state.selected_historic_date = date
                st.session_state.manual_override_price = None
                write_price_to_stage(price, False)
                st.rerun()
    
    st.divider()
    
    # Manual price input section
    st.write("**Manual Price Entry:**")
    
    # Determine if field should be disabled (greyed out when historic is selected)
    is_field_disabled = st.session_state.selected_historic_date is not None
    
    # Set value to manual price if set, otherwise 0
    if st.session_state.manual_override_price is not None:
        default_val = st.session_state.manual_override_price
    else:
        default_val = 0.0
    
    col_input, col_button = st.columns([3, 1])
    
    with col_input:
        new_price = st.number_input(
            "Enter current price (€/tCO₂e)", 
            min_value=0.0, 
            max_value=500.0,
            value=float(default_val),
            step=0.5,
            help="Enter a custom carbon price for calculations",
            key="price_input",
            disabled=is_field_disabled
        )
    
    with col_button:
        st.write("")  # Spacing
        st.write("")  # Spacing
        if st.button("✅ Update Price", use_container_width=True, disabled=is_field_disabled):
            if new_price > 0:
                st.session_state.carbon_price = new_price
                st.session_state.manual_override_price = new_price
                st.session_state.selected_historic_date = None
                write_price_to_stage(new_price, True)
                st.rerun()
            else:
                st.warning("Please enter a price greater than 0")
    
    st.markdown("---")
    
    # Emission factors reference
    with st.expander("📋 Default Emission Factors"):
        st.write("**Default Embedded Emissions (tCO₂e per tonne of product)**")
        em_df = pd.DataFrame([
            {"Product": k.title(), "Emission Factor (tCO₂e/tonne)": v}
            for k, v in DEFAULT_EMISSIONS.items()
        ])
        st.dataframe(em_df, hide_index=True, use_container_width=True)
        st.caption("These are used when specific emission data is not provided")
    
    # Display chat history
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

def get_chat_history():
    """Get recent chat history for context."""
    start_index = max(0, len(st.session_state.messages) - st.session_state.num_chat_messages)
    return st.session_state.messages[start_index : len(st.session_state.messages) - 1]

def extract_cbam_request(question):
    """Extract CBAM calculation parameters from user question."""
    match = re.search(r"(\d+(?:,\d{3})*)\s+tons?(?:\s+of)?\s+(\w+)", question, re.IGNORECASE)
    emissions_match = re.search(r"(\d+(?:\.\d+)?)\s*tCO2e?(?:/ton)?", question, re.IGNORECASE)
    origin_price_match = re.search(r"(?:origin|paid|cost).*?€?(\d+(?:\.\d+)?)", question, re.IGNORECASE)

    if match:
        quantity_str = match.group(1).replace(",", "")
        quantity = int(quantity_str)
        product = match.group(2).lower()
    else:
        quantity = None
        product = None

    emissions = float(emissions_match.group(1)) if emissions_match else None
    origin_price = float(origin_price_match.group(1)) if origin_price_match else 0.0
    
    return product, quantity, emissions, origin_price

def calculate_cbam_cost(embedded_emissions, origin_carbon_price=0, eu_carbon_price=None):
    """Calculate CBAM cost based on emissions and carbon prices."""
    if eu_carbon_price is None:
        eu_carbon_price = st.session_state.carbon_price
    
    cbam_cost = embedded_emissions * (eu_carbon_price - origin_carbon_price)
    return max(0, cbam_cost)

def cortex_search(my_question):
    """Search documentation using Cortex Search Service."""
    try:
        search_service = (root
            .databases[database_name]
            .schemas[schema_name]
            .cortex_search_services[service_name]
        )
        resp = search_service.search(
            query=my_question,
            columns=["text", "file_name"],
            limit=num_results
        )
        results = json.loads(resp.to_json())["results"]
        prompt_context = "\n\n".join([r["text"] for r in results]).replace("'", "")
        file_name = results[0]['file_name'] if results else "No source"
        return prompt_context[:8000], file_name
    except Exception as e:
        st.error(f"Search failed: {str(e)}")
        return "", "Error"

def format_chat_history(chat_history):
    """Format chat history for prompt context."""
    return "\n".join([f"{m['role']}: {m['content']}" for m in chat_history[-3:]])

def create_prompt(user_question):
    """Create prompt for LLM with context and instructions."""
    chat_history = get_chat_history()
    history_str = format_chat_history(chat_history)
    prompt_context, file_name = cortex_search(user_question)
    
    current_price = st.session_state.carbon_price

    prompt = f"""You are a CBAM (Carbon Border Adjustment Mechanism) specialist. Provide direct, concise answers using the provided documentation.

<context>
{prompt_context}

Current EU ETS Carbon Price: €{current_price:.2f}/tonne CO₂e
</context>

<chat_history>
{history_str}
</chat_history>

<question>
{user_question}
</question>

<instructions>
1. Answer directly from the provided documentation and current carbon price data.
2. Cite specific values, formulas, and guidance from the documents.
3. For CBAM calculations:
   - Use default emission factors if actual emissions not provided: {DEFAULT_EMISSIONS}
   - Formula: CBAM Cost = (Embedded Emissions tCO₂e) × (EU ETS Price €{current_price:.2f} - Carbon Price Paid in Origin)
   - Always show calculation steps clearly
4. Keep responses under 150 words unless calculations require detail
5. Structure: Direct answer → Key requirements → Calculations (if applicable) → Limitations (if any)
6. If information is missing: State what's needed clearly
7. Be precise about deadlines, compliance requirements, and reporting obligations
</instructions>

Response:"""
    return prompt, file_name

def complete_with_retry(model, prompt, retries=2):
    """Call LLM completion with rate limiting and retry logic."""
    now = time.time()
    if now - st.session_state.last_request_time < MIN_REQUEST_INTERVAL:
        time.sleep(MIN_REQUEST_INTERVAL - (now - st.session_state.last_request_time))

    for attempt in range(retries):
        try:
            response = complete(model, prompt)
            st.session_state.last_request_time = time.time()
            return response
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(RETRY_DELAY * (attempt + 1))
            else:
                st.error(f"Request failed: {str(e)}")
                return None

def main():
    st.title("🌍 CBAM Calculator & Documentation Assistant")
    st.caption("Calculate Carbon Border Adjustment Mechanism costs with EU ETS pricing")
    
    init_messages()
    init_config_options()
    
    icons = {"assistant": "❄️", "user": "👤"}

    if question := st.chat_input("Ask about CBAM calculations, emissions, or requirements..."):
        st.session_state.messages.append({"role": "user", "content": question})

        with st.chat_message("user", avatar=icons["user"]):
            st.markdown(question)

        # Check for direct calculation request
        product, quantity, emissions, origin_price = extract_cbam_request(question)
        if product and quantity:
            if emissions is None:
                emissions = DEFAULT_EMISSIONS.get(product)
            
            if emissions is not None:
                total_emissions = emissions * quantity
                cbam_cost = calculate_cbam_cost(total_emissions, origin_price)
                
                eu_price = st.session_state.carbon_price
                
                response_text = f"""💶 **CBAM Cost Calculation**

**Product:** {product.title()}  
**Quantity:** {quantity:,} tonnes  
**Emission Factor:** {emissions} tCO₂e/tonne  
**Total Emissions:** {total_emissions:,.2f} tCO₂e  
**EU ETS Price:** €{eu_price:.2f}/tCO₂e  
**Origin Carbon Price:** €{origin_price:.2f}/tCO₂e  

---

**💰 Estimated CBAM Cost: €{cbam_cost:,.2f}**

---

*Calculation: {total_emissions:,.2f} tCO₂e × (€{eu_price:.2f} - €{origin_price:.2f}) = €{cbam_cost:,.2f}*

📝 Note: This is an estimate. Actual CBAM liability depends on verified emissions data and official EU ETS prices at the time of import.
"""
                st.session_state.messages.append({"role": "assistant", "content": response_text})
                with st.chat_message("assistant", avatar=icons["assistant"]):
                    st.markdown(response_text)
                return

        # Handle documentation questions
        with st.chat_message("assistant", avatar=icons["assistant"]):
            with st.spinner("Analyzing documentation..."):
                prompt, file_name = create_prompt(question)
                response = complete_with_retry(model_name, prompt)
                if response:
                    st.markdown(response)
                    st.caption(f"📄 Source: {file_name}")
                    st.session_state.messages.append({"role": "assistant", "content": response})

if __name__ == "__main__":
    main()